# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.6.0


# Check point function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# data load & preprocessing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

# model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

# Create model -- class version

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# class model -- function verstion

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data and hyperparameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# define model , optimizer, writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore Checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.7512, train_loss: 0.32009947, train_accuracy: 0.3750, test_Accuracy: 0.2275
Epoch: [ 0] [    1/  468] time: 1.0724, train_loss: 0.35834432, train_accuracy: 0.3516, test_Accuracy: 0.3481
Epoch: [ 0] [    2/  468] time: 1.4021, train_loss: 0.33815801, train_accuracy: 0.4922, test_Accuracy: 0.4506
Epoch: [ 0] [    3/  468] time: 1.7791, train_loss: 0.31599411, train_accuracy: 0.5781, test_Accuracy: 0.5272
Epoch: [ 0] [    4/  468] time: 2.1352, train_loss: 0.33043876, train_accuracy: 0.6406, test_Accuracy: 0.5766
Epoch: [ 0] [    5/  468] time: 2.5124, train_loss: 0.28999853, train_accuracy: 0.6406, test_Accuracy: 0.6105
Epoch: [ 0] [    6/  468] time: 2.9155, train_loss: 0.30134916, train_accuracy: 0.6484, test_Accuracy: 0.6384
Epoch: [ 0] [    7/  468] time: 3.3144, train_loss: 0.21224853, train_accuracy: 0.6641, test_Accuracy: 0.6631
Epoch: [ 0] [    8/  468] time: 3.7317,

Epoch: [ 0] [   74/  468] time: 29.6655, train_loss: 0.06300708, train_accuracy: 0.9375, test_Accuracy: 0.9381
Epoch: [ 0] [   75/  468] time: 30.1204, train_loss: 0.15435430, train_accuracy: 0.9453, test_Accuracy: 0.9397
Epoch: [ 0] [   76/  468] time: 30.5608, train_loss: 0.12232414, train_accuracy: 0.9531, test_Accuracy: 0.9402
Epoch: [ 0] [   77/  468] time: 30.9856, train_loss: 0.13781936, train_accuracy: 0.9531, test_Accuracy: 0.9402
Epoch: [ 0] [   78/  468] time: 31.4075, train_loss: 0.18393767, train_accuracy: 0.9141, test_Accuracy: 0.9408
Epoch: [ 0] [   79/  468] time: 31.8351, train_loss: 0.17278197, train_accuracy: 0.9453, test_Accuracy: 0.9416
Epoch: [ 0] [   80/  468] time: 32.2680, train_loss: 0.12843841, train_accuracy: 0.9609, test_Accuracy: 0.9409
Epoch: [ 0] [   81/  468] time: 32.6813, train_loss: 0.18646964, train_accuracy: 0.9453, test_Accuracy: 0.9400
Epoch: [ 0] [   82/  468] time: 33.0873, train_loss: 0.14519429, train_accuracy: 0.9609, test_Accuracy: 0.9385
E

Epoch: [ 0] [  148/  468] time: 60.0369, train_loss: 0.11865445, train_accuracy: 0.9453, test_Accuracy: 0.9480
Epoch: [ 0] [  149/  468] time: 60.4296, train_loss: 0.08050548, train_accuracy: 0.9453, test_Accuracy: 0.9500
Epoch: [ 0] [  150/  468] time: 60.8083, train_loss: 0.05491754, train_accuracy: 0.9766, test_Accuracy: 0.9506
Epoch: [ 0] [  151/  468] time: 61.2227, train_loss: 0.14539196, train_accuracy: 0.9453, test_Accuracy: 0.9505
Epoch: [ 0] [  152/  468] time: 61.6490, train_loss: 0.09993868, train_accuracy: 0.9766, test_Accuracy: 0.9519
Epoch: [ 0] [  153/  468] time: 62.0464, train_loss: 0.06375684, train_accuracy: 0.9531, test_Accuracy: 0.9520
Epoch: [ 0] [  154/  468] time: 62.4427, train_loss: 0.11853626, train_accuracy: 0.9531, test_Accuracy: 0.9515
Epoch: [ 0] [  155/  468] time: 62.8243, train_loss: 0.11892506, train_accuracy: 0.9766, test_Accuracy: 0.9521
Epoch: [ 0] [  156/  468] time: 63.2383, train_loss: 0.20783556, train_accuracy: 0.9375, test_Accuracy: 0.9518
E

Epoch: [ 0] [  222/  468] time: 89.1679, train_loss: 0.10670909, train_accuracy: 0.9688, test_Accuracy: 0.9606
Epoch: [ 0] [  223/  468] time: 89.5583, train_loss: 0.09456630, train_accuracy: 0.9844, test_Accuracy: 0.9593
Epoch: [ 0] [  224/  468] time: 89.9682, train_loss: 0.08632980, train_accuracy: 0.9688, test_Accuracy: 0.9585
Epoch: [ 0] [  225/  468] time: 90.4001, train_loss: 0.14247277, train_accuracy: 0.9609, test_Accuracy: 0.9570
Epoch: [ 0] [  226/  468] time: 90.7834, train_loss: 0.05724812, train_accuracy: 0.9766, test_Accuracy: 0.9558
Epoch: [ 0] [  227/  468] time: 91.1501, train_loss: 0.10306007, train_accuracy: 0.9844, test_Accuracy: 0.9549
Epoch: [ 0] [  228/  468] time: 91.5254, train_loss: 0.03494722, train_accuracy: 0.9766, test_Accuracy: 0.9534
Epoch: [ 0] [  229/  468] time: 91.9224, train_loss: 0.13775726, train_accuracy: 0.9453, test_Accuracy: 0.9515
Epoch: [ 0] [  230/  468] time: 92.3002, train_loss: 0.06010703, train_accuracy: 0.9531, test_Accuracy: 0.9498
E

Epoch: [ 0] [  296/  468] time: 118.1336, train_loss: 0.07807847, train_accuracy: 0.9844, test_Accuracy: 0.9631
Epoch: [ 0] [  297/  468] time: 118.5221, train_loss: 0.07500529, train_accuracy: 0.9609, test_Accuracy: 0.9641
Epoch: [ 0] [  298/  468] time: 118.9044, train_loss: 0.09353110, train_accuracy: 0.9688, test_Accuracy: 0.9651
Epoch: [ 0] [  299/  468] time: 119.2904, train_loss: 0.03936277, train_accuracy: 0.9844, test_Accuracy: 0.9648
Epoch: [ 0] [  300/  468] time: 119.6822, train_loss: 0.06878979, train_accuracy: 0.9609, test_Accuracy: 0.9655
Epoch: [ 0] [  301/  468] time: 120.0771, train_loss: 0.12307705, train_accuracy: 0.9531, test_Accuracy: 0.9660
Epoch: [ 0] [  302/  468] time: 120.4581, train_loss: 0.06044445, train_accuracy: 0.9766, test_Accuracy: 0.9652
Epoch: [ 0] [  303/  468] time: 120.8485, train_loss: 0.05839624, train_accuracy: 0.9766, test_Accuracy: 0.9646
Epoch: [ 0] [  304/  468] time: 121.2505, train_loss: 0.17038046, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 147.0434, train_loss: 0.09702756, train_accuracy: 0.9766, test_Accuracy: 0.9645
Epoch: [ 0] [  371/  468] time: 147.4449, train_loss: 0.02010476, train_accuracy: 1.0000, test_Accuracy: 0.9645
Epoch: [ 0] [  372/  468] time: 147.8417, train_loss: 0.18649700, train_accuracy: 0.9531, test_Accuracy: 0.9658
Epoch: [ 0] [  373/  468] time: 148.2193, train_loss: 0.04666519, train_accuracy: 0.9766, test_Accuracy: 0.9668
Epoch: [ 0] [  374/  468] time: 148.6034, train_loss: 0.09525251, train_accuracy: 0.9453, test_Accuracy: 0.9666
Epoch: [ 0] [  375/  468] time: 149.0058, train_loss: 0.08761846, train_accuracy: 0.9844, test_Accuracy: 0.9662
Epoch: [ 0] [  376/  468] time: 149.4033, train_loss: 0.09043463, train_accuracy: 0.9766, test_Accuracy: 0.9665
Epoch: [ 0] [  377/  468] time: 149.8072, train_loss: 0.15825650, train_accuracy: 0.9531, test_Accuracy: 0.9652
Epoch: [ 0] [  378/  468] time: 150.2057, train_loss: 0.09460758, train_accuracy: 0.9844, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 175.9767, train_loss: 0.14359978, train_accuracy: 0.9453, test_Accuracy: 0.9678
Epoch: [ 0] [  445/  468] time: 176.3297, train_loss: 0.11540407, train_accuracy: 0.9609, test_Accuracy: 0.9681
Epoch: [ 0] [  446/  468] time: 176.6974, train_loss: 0.06680116, train_accuracy: 0.9844, test_Accuracy: 0.9684
Epoch: [ 0] [  447/  468] time: 177.0882, train_loss: 0.07602669, train_accuracy: 0.9766, test_Accuracy: 0.9691
Epoch: [ 0] [  448/  468] time: 177.4658, train_loss: 0.06503405, train_accuracy: 0.9766, test_Accuracy: 0.9697
Epoch: [ 0] [  449/  468] time: 177.8477, train_loss: 0.06992680, train_accuracy: 0.9766, test_Accuracy: 0.9697
Epoch: [ 0] [  450/  468] time: 178.2413, train_loss: 0.01683348, train_accuracy: 0.9922, test_Accuracy: 0.9683
Epoch: [ 0] [  451/  468] time: 178.6271, train_loss: 0.09530995, train_accuracy: 0.9844, test_Accuracy: 0.9681
Epoch: [ 0] [  452/  468] time: 179.0221, train_loss: 0.04886873, train_accuracy: 0.9922, test_Accuracy: